In [1]:
#!pip3 install ppi_client

In [1]:
from datetime import datetime, timedelta

from ppi_client.api.constants import ACCOUNTDATA_TYPE_ACCOUNT_NOTIFICATION, ACCOUNTDATA_TYPE_PUSH_NOTIFICATION, \
    ACCOUNTDATA_TYPE_ORDER_NOTIFICATION
from ppi_client.models.account_movements import AccountMovements
from ppi_client.models.bank_account_request import BankAccountRequest
from ppi_client.models.foreign_bank_account_request import ForeignBankAccountRequest, ForeignBankAccountRequestDTO
from ppi_client.models.cancel_bank_account_request import CancelBankAccountRequest
from ppi_client.models.order import Order
from ppi_client.ppi import PPI
from ppi_client.models.order_budget import OrderBudget
from ppi_client.models.order_confirm import OrderConfirm
from ppi_client.models.disclaimer import Disclaimer
from ppi_client.models.investing_profile import InvestingProfile
from ppi_client.models.investing_profile_answer import InvestingProfileAnswer
from ppi_client.models.instrument import Instrument
from ppi_client.models.estimate_bonds import EstimateBonds
import asyncio
import json
import traceback
import os
import numpy as np
import os
from dotenv import load_dotenv
from google.cloud import bigquery

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\ffliguer\Desktop\Recursos\scraping-256217-ccd9f463218c.json'

In [1]:
def insert_rows(name_table, dict_row):
    client = bigquery.Client()
    table_id = bigquery.Table(name_table)
    rows_to_insert = [dict_row]
    errors = client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    if errors == []:
        msg = "New rows have been added."
        #print("New rows have been added.")
    else:
        print("Encountered errors while inserting rows: {}".format(errors))

def calculador_potencial(ticker, tipo_ticker, tipo_output, valor_input, side, a_dolar = False, conn = False):
    ppi = conn
    if ticker == 'GOOGL' and a_dolar:
        ticker = 'GOGLD'
    elif tipo_ticker == 'Cedears' and a_dolar:
        ticker = ticker + 'D'
    try:
        current_book = ppi.marketdata.book(ticker, tipo_ticker, "A-48HS")
    except:
        raise ValueError("No se encuentra el libro del activo.")
    libro_q = [orden['quantity'] for orden in current_book[side]]
    libro_p = [orden['price'] for orden in current_book[side]]
    if len(libro_q) == 0 or len(libro_p) == 0:
        raise ValueError("No hay órdenes vigentes.")
    if tipo_output == 'ppp':
        if np.sum(libro_q) < valor_input:
            raise ValueError("Las órdenes vigentes no alcanzan para la operación que se busca hacer.")
        libro_cum_q = np.cumsum(libro_q)
        nu_ordenes_necesarias = [n for n,i in enumerate(libro_cum_q) if i >= valor_input][0]
        q_ordenes_completas = sum(libro_q[0:nu_ordenes_necesarias])
        ppp_ordenes_completas = np.array(libro_q[0:nu_ordenes_necesarias]) @ np.array(libro_p[0:nu_ordenes_necesarias])
        q_orden_incompleta = valor_input - q_ordenes_completas
        ppp_orden_incompleta = libro_p[nu_ordenes_necesarias] * q_orden_incompleta
        ppp = ppp_ordenes_completas + ppp_orden_incompleta
        return ppp
    elif tipo_output == 'qqq':
        libro_pq = np.multiply(libro_q, libro_p)
        libro_cum_pq = np.cumsum(libro_pq)
        if np.sum(libro_pq) < valor_input:
            raise ValueError("Las órdenes vigentes no alcanzan para la operación que se busca hacer.")
        nu_ordenes_necesarias = [n for n,i in enumerate(libro_cum_pq) if i >= valor_input][0]
        pq_ordenes_completas = sum(libro_pq[0:nu_ordenes_necesarias])
        q_ordenes_completas = sum(libro_q[0:nu_ordenes_necesarias])
        pq_orden_incompletas = valor_input - pq_ordenes_completas
        q_orden_incompleta = pq_orden_incompletas / libro_p[nu_ordenes_necesarias]
        qqq = q_ordenes_completas + q_orden_incompleta
        return qqq
    
def main():
    ppi = PPI(sandbox=False)
    load_dotenv(r'C:\Users\ffliguer\Desktop\Proyectos\Market projects\Arbitraje de MEP via CEDEARS en PPI (b1)\.env')
    key_publica = os.getenv('key_publica')
    key_privada = os.getenv('key_privada')
    account_number = os.getenv('account_number')
    ppi.account.login_api(key_publica, key_privada)
    
    current_date = datetime.now()
    id_ex = current_date.year*10000000000 + current_date.month * 100000000 + current_date.day * 1000000 + current_date.hour*10000 + current_date.minute * 100 + current_date.second

    tabla = 'scraping-256217.bots.b1_evaluaciones'
    balances_positions = ppi.account.get_balance_and_positions(account_number)
    for instruments in balances_positions["groupedInstruments"]:
        for instrument in instruments['instruments']:
            for bono in ['AL30', 'GD30']:
                t, q, ppp_usd, nominales_bono_d, ppp_bono, nominales_potenciales_recupero  = None, None, None, None, None, None
                ppp_usd
                print('Con ', bono)
                try:
                    t = instrument['ticker']
                    q = instrument['amount'] / instrument['price']
                    print('TICKER:', t)
                    print(' POSICIÓN:', q)
                    ppp_usd = calculador_potencial(t, 'Cedears', 'ppp', q, side='bids', a_dolar = True, conn = ppi)
                    print(' DOLARES POTENCIALES:', round(ppp_usd,2))
                    nominales_bono_d = calculador_potencial(bono + 'D', 'Bonos', 'qqq', ppp_usd, side='offers', a_dolar = False, conn = ppi)
                    print(' NOMINALES DE', bono, 'POTENCIALES:', nominales_bono_d)
                    ppp_bono = calculador_potencial(bono, 'Bonos', 'ppp', nominales_bono_d, side='bids', a_dolar = False, conn = ppi)
                    print(' PESOS POR ESOS NOMINALES:', ppp_bono)
                    nominales_potenciales_recupero = calculador_potencial(t, 'Cedears', 'qqq', ppp_bono, side='offers', a_dolar = False, conn = ppi)
                    print(' CANTIDAD DEL TICKER QUE PODRÍA RECOMPRAR:', nominales_potenciales_recupero)
                except:
                    print('Al menos una operación no puede realizarse.')

                try:
                    ratio = nominales_potenciales_recupero/q
                except:
                    ratio = None
                nueva_fila = {"id_bot": 1, 
                            "id_ejecucion": id_ex, 
                            "time": datetime.timestamp(datetime.now()), 
                            "ticker": t,
                            "posicion": q,
                            "dolares_potenciales": ppp_usd,
                            "bono": bono,
                            "nominales_potenciales": nominales_bono_d,
                            "pesos_potenciales": ppp_bono,
                            "nueva_posicion_potencial": nominales_potenciales_recupero,
                            "ratio": ratio}
                insert_rows(tabla, nueva_fila)
    return 'Fin de la ejecución'